Pipeline for Extracting Keywords from the dataset

In [84]:
import pandas as pd
import spacy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np
#import nltk
#nltk.download('punkt')


In [85]:
nlp = spacy.load("en_core_web_sm")

path = "/Users/ethanvirtudazo/Desktop/DS105_Dataset/6622_jobs.xls"
df_import = pd.read_excel(path)

df_import.shape

(6623, 11)

In [86]:
df_import.head(2)

,Unnamed: 0,title,details,deadline,opport_type,commence_date,contract_type,location,Renumeration,company,links
0,0,Rothschild & Co - Private Equity Long-Term Int...,This London-based 6-month internship is an exc...,2023-04-30,Internship,2023-07-01 00:00:00,Temporary,London,NaN,Rothschild & Co,https://careers.lse.ac.uk//students/jobs/detai...
1,1,2023 HSBC Global Graduate Programme (Hong Kong...,You’re excited about starting your career and ...,2023-01-06,Graduate employment,2023-07-03 00:00:00,Temporary,Hong KongSingapore,NaN,HSBC (HSBC) - Hong Kong,https://careers.lse.ac.uk//students/jobs/detai...


In [87]:
df1 = df_import.iloc[:, 1:5]
df1.head()

df2 = df_import.iloc[:, [7,9,10]]
df2.head()

pdList = [df1,df2]
df = pd.concat(pdList,axis=1)
df.head()

df.shape

types = df.dtypes
print(types)

df_title = df.iloc[:,0]
df_text_both = df.iloc[:, [0,1]]
df_text = df.iloc[:, [1]]

df_text_both.head()


title                  object
details                object
deadline       datetime64[ns]
opport_type            object
location               object
company                object
links                  object
dtype: object


,title,details
0,Rothschild & Co - Private Equity Long-Term Int...,This London-based 6-month internship is an exc...
1,2023 HSBC Global Graduate Programme (Hong Kong...,You’re excited about starting your career and ...
2,2023 HSBC Global Internship Programme (Hong Ko...,You’re excited about starting your career and ...
3,"Graduate Training Scheme, Capital Markets",Graduate Training Scheme – LondonGreySpark Par...
4,6-Months Internship – Sell-side Tech M&A,"At IPTP, we understand software from decades o..."


In [88]:
df_text.head()

,details
0,This London-based 6-month internship is an exc...
1,You’re excited about starting your career and ...
2,You’re excited about starting your career and ...
3,Graduate Training Scheme – LondonGreySpark Par...
4,"At IPTP, we understand software from decades o..."


In [89]:
type(df_text)

pandas.core.frame.DataFrame

In [90]:
#ar_text = df_text.to_numpy()
#ar_text = ar_text[:101]

df_text_1 = df_text[:101]
len(df_text_1)

101

In [93]:
# Converting text data of the sample data into nlp object called 'document'

#txt_lst = df_text['details'].tolist()

docs_1 = df_text_1['details'].apply(nlp)
len(docs_1)

101

In [95]:
# Viewing specific nlp doc
docs_1[0]

This London-based 6-month internship is an exciting opportunity to intern with Rothschild & Co's European corporate private equity business. Rothschild & Co’s European corporate private equity business is comprised of three strategies: Five Arrows Principal Investments (FAPI), Five Arrows Growth Capital (FAGC), and Five Arrows Long Term Fund (FALT). FAPI manages c. €1.3 billion through its latest fund and is the flagship corporate private equity / buy-out vehicle investing in mid-cap companies, while FAGC manages c. €500 million through growth investments and small-cap buy-outs, and FALT makes investments in larger companies with the option for longer-term holding periods. All three funds are focused on investments within Western Europe (with FALT additionally on North America) and primarily invest in three main verticals: Data & Software, Healthcare, and Technology-enabled Business Services. The respective teams are based in London, Paris and Luxembourg and are comprised of c. 30 inve

In [96]:
#Individual Job View of NER 
# NER = Name Entity Recognition
rend_doc_1 = docs_1[0]

from spacy import displacy
displacy.render(rend_doc_1,style="ent")

In [97]:
filtered_docs_1 = list([[token.text for token in doc if token.pos_ in ['PROPN', 'NOUN','ADJ']] for doc in docs_1])
len(filtered_docs_1)

101

In [98]:
# words that are not proper nouns, nouns, or adjectives are removed from the documents.
filtered_docs_1[0]

['London',
 'month',
 'internship',
 'exciting',
 'opportunity',
 'Rothschild',
 'Co',
 'European',
 'corporate',
 'private',
 'equity',
 'business',
 'Rothschild',
 'Co',
 'European',
 'corporate',
 'private',
 'equity',
 'business',
 'strategies',
 'Arrows',
 'Principal',
 'Investments',
 'FAPI',
 'Arrows',
 'Growth',
 'Capital',
 'FAGC',
 'Arrows',
 'Long',
 'Term',
 'Fund',
 'FALT',
 'FAPI',
 'c.',
 'latest',
 'fund',
 'flagship',
 'corporate',
 'private',
 'equity',
 'vehicle',
 'mid',
 '-',
 'cap',
 'companies',
 'FAGC',
 'c.',
 'growth',
 'investments',
 'small',
 'cap',
 'buy',
 'outs',
 'FALT',
 'investments',
 'larger',
 'companies',
 'option',
 'longer',
 'term',
 'holding',
 'periods',
 'funds',
 'investments',
 'Western',
 'Europe',
 'FALT',
 'North',
 'America',
 'main',
 'verticals',
 'Data',
 'Software',
 'Healthcare',
 'Technology',
 'Business',
 'Services',
 'respective',
 'teams',
 'London',
 'Paris',
 'Luxembourg',
 'c.',
 'investment',
 'professionals',
 'countries

In [99]:
# join all tokens for each job into a single string
fil_lst_1 = [' '.join(lst) for lst in filtered_docs]
len(fil_lst_1)

101

In [100]:
fil_lst_1[0]

'London month internship exciting opportunity Rothschild Co European corporate private equity business Rothschild Co European corporate private equity business strategies Arrows Principal Investments FAPI Arrows Growth Capital FAGC Arrows Long Term Fund FALT FAPI c. latest fund flagship corporate private equity vehicle mid - cap companies FAGC c. growth investments small cap buy outs FALT investments larger companies option longer term holding periods funds investments Western Europe FALT North America main verticals Data Software Healthcare Technology Business Services respective teams London Paris Luxembourg c. investment professionals countries Responsibilities industries potential targets investments sources information Identifying investment opportunities financial models investment committee memos standard financial models deal execution processes monitoring portfolio companies point view value actionable opportunities Fund risk return criteria team co - due diligence transaction

In [101]:
# CREATING DATAFRAME FOR R

# 1 Turning titles 
df_title_r_1 = df_title[0:101]
df_title_r_1 = pd.DataFrame(df_title_r)
type(df_title_r_1)

pandas.core.frame.DataFrame

In [102]:
df_title_r_1.head()

,title
0,Rothschild & Co - Private Equity Long-Term Int...
1,2023 HSBC Global Graduate Programme (Hong Kong...
2,2023 HSBC Global Internship Programme (Hong Ko...
3,"Graduate Training Scheme, Capital Markets"
4,6-Months Internship – Sell-side Tech M&A


In [104]:
df_fil_1 = pd.DataFrame(fil_lst_1)
type(df_fil_1)

pandas.core.frame.DataFrame

In [105]:
len(df_fil_1)

101

In [107]:
# CREATING DATAFRAME FOR R

df_r_1 = pd.concat([df_title_r_1, df_fil_1], axis = 1)
df_r_1

,title,0
0,Rothschild & Co - Private Equity Long-Term Int...,London month internship exciting opportunity R...
1,2023 HSBC Global Graduate Programme (Hong Kong...,excited career many paths possibilities global...
2,2023 HSBC Global Internship Programme (Hong Ko...,excited career many paths possibilities global...
3,"Graduate Training Scheme, Capital Markets",Graduate Training Scheme LondonGreySpark Partn...
4,6-Months Internship – Sell-side Tech M&A,IPTP software decades deep experience technolo...
...,...,...
96,M&A Analyst Intern,MAJOR RESPONSIBILITIESGather financial operat...
97,12 Month Internship - Financial Crimes and San...,Job summaryFinancial Crime Financial Security ...
98,Investment Associate - Fixed Income,Position OverviewPutnam energetic curious indi...
99,12 Month Internship - Central Compliance,SummaryThe Central Compliance team responsible...


In [108]:
#Export as .csv file
df_r_1.to_csv('df_r_1.csv')

Below is the code for applying the pipeline to the entire dataset

1. Cleaning the data 
-identify index of non-text data
-removing nan (non-text) data

2. Convert to nlp object 'document'


3. 